In [2]:
from app.tools.functions import *
from app.tools.constans import *
from app.tools.db import  *
from app.tools.s3 import *
from app.match import *
import pandas as pd


conn = DatabaseConnection(**paramsl)

/home/lharo/programing/match2/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
request = {
    "request_id": "e07aaf8f-a587-42af-a0b3-0abd42a7ffc5",
    "input":{
                "origin" : {"products":{"brand": "3PIN"}},
                "alternative" : {"products":{"brand": "3PIN"}}
            },
    "s3_path_img_origin": "ajio-myntra/origin/20240131/",
    "s3_path_img_alternative": ["ajio-myntra/alternative/20240131/dev/", "ajio-myntra/alternative/20240202/dev/", "ajio-myntra/alternative/20240201/dev/"]
}

# match_img(**request)